# River masks Semi-authomatic Multi-Temporal extraction from Landsat SR images usign Google Earth Engine
### Variant of the original code that uses a service account

### Niccolò Ragno, Riccardo Bonanomi, Marta Crivellaro, Alfonso Vitti, Guido Zolezzi and Marco Tubino
* Publication corresponding Author: Niccolò Ragno,  niccolo.ragno@unitn.it
* GEE code corresponding Author: Marta Crivellaro,  marta.crivellaro@unitn.it

GEE Python installation: https://developers.google.com/earth-engine/guides/python_install

In [ ]:
#libraries import
import os
import pandas as pd
from functions import areaImg, ndviMap, mndwiMap, histogram, otsu, peronaMalikFilter, check_tasks_status
import ee
import geemap
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials
import time

In [ ]:
# Google Earth Engine authentication - see readme for details
from credentials_script import get_credentials
service_account, credentials_folder, credentials_file = get_credentials()
credentials = ee.ServiceAccountCredentials(service_account, credentials_folder + credentials_file)
ee.Initialize(credentials)

# Google drive autentiucation - see readme for details
gauth = GoogleAuth()
scopes = ['https://www.googleapis.com/auth/drive']
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_folder+credentials_file, scopes=scopes)
drive = GoogleDrive(gauth)

In [ ]:
# import input data from function - see readme for details
from input_data import get_input_data
river_name, source, scaledLS, print_errors, delete_other, download_wait, coordinates, EPSG_code, dates, output_folder = get_input_data() # import the input data

#1 - Define region(S) of interest (roi) as rectangular extents
river_geom = ee.Geometry.Polygon(coordinates)

# output data folders
# local
os.makedirs(output_folder, exist_ok=True)

# Google Drive
drive_folder = river_name + '-' + 'Permafrost_rivermask_' + source

### Create an interactive map
The default basemap is _Google Maps_. Additional basemaps can be added using the ``Map.add_basemap()`` function.

In [ ]:
Map = geemap.Map(center=[19.76,40.41], zoom=22)
print('Done!')

In [ ]:
fcGeom = river_geom
roi = fcGeom 
Map.centerObject(roi)
Map

Standardise band names, merge Landsat data:

In [ ]:
if source == 'ls':
    # LANDSAT source
    # Standardise band names, merge Landsat data:
    bn8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B6', 'SR_QA_AEROSOL', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    bn7 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B5', 'SR_CLOUD_QA'  , 'SR_B4', 'SR_B7', 'QA_PIXEL']
    bn5 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B5', 'SR_CLOUD_QA'  , 'SR_B4', 'SR_B7', 'QA_PIXEL']

    #standard bands:
    bnL = ['uBlue', 'Blue' , 'Green', 'Red'  , 'Swir1', 'BQA'          , 'Nir'  , 'Swir2', 'qa_pixel']

    scale      = 30      # pixel size of Landsat images in meters
    scale_hist = scale/2 # pixel size for the histogram calculation in meters
    radio_res  = 65455

    ## defining cloudmask function for landsat 7 and 8 only
    # This function masks the input with a threshold on the simple cloud score.
    # Observe that the input to simpleCloudScore() is a single Landsat TOA scene. 
    # Also note that simpleCloudScore() adds a band called ‘cloud’ to the input image. 
    # The cloud band contains the cloud score from 0 (not cloudy) to 100 (most cloudy).
    def cloudMask(img):
        cloudscore = ee.Algorithms.Landsat.simpleCloudScore(img).select('cloud')
        return img.updateMask(cloudscore.lt(10))

    def maskClouds(image):
        
        cloudShadowBitMask = (1 << 3)
        cloudsBitMask = (1 << 5)
        
        qa = image.select('qa_pixel')
        mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0)))
        
        return image.updateMask(mask)

    # calling LS Surface Reflectance image collections 
    ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filter(ee.Filter.lt('CLOUD_COVER',15)).select(bn5, bnL)
    ls7 = (ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")  \
    .map(cloudMask)  \
    .filterDate('1999-04-15', '2003-05-30')  \
    .select(bn7, bnL))
    ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(ee.Filter.lt('CLOUD_COVER', 15)).select(bn8, bnL)
    # merging LS 5 and 8 dataset
    ls = ls5.merge(ls8).sort('system:start', True).filter(ee.Filter.calendarRange(5,9,'month'))

elif source == 's2':
    # SENTINEL-2 data
    # Standardise band names, merge Sentinel data:
    bnS1 = ['B1', 'B2', 'B3', 'B4', 'B6', 'B5', 'B7','B8','B8A','B9','B11','B12','QA60']
    # standard bands:
    bns1 = ['Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1', 'RedEdge2','RedEdge3','Nir','RedEdge4', 'WaterVapor','Swir1','Swir2','cldmask']

    scale      = 10    # pixel size of Landsat images in meters
    scale_hist = scale # pixel size for the histogram calculation in meters
    radio_res  = 10000
    
    # calling Sentinel-2 MSI: MultiSpectral Instrument, Level-2A

    def maskCloudsS2(image):
        cloudsBitMask = (1 << 10)
        cirrusBitMask = (1 << 11)
        qa = image.select('cldmask')
        mask = (qa.bitwiseAnd(cloudsBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)))
        return image.updateMask(mask)

    s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").select(bnS1, bns1)\
                 .sort('system:start', True).filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',15).map(maskCloudsS2).filter(ee.Filter.calendarRange(5,9,'month'))
    
else:
    raise ValueError("data must be Landasat 'ls' or Sentinel-2 's2'")

In [ ]:
#functions definition
def RGBtoHSV (Image):
    sat = Image.select(['Red','Green','Blue']).divide(radio_res).rgbToHsv().select(['saturation'])
    return Image.addBands(sat)

### Cycle on years to export annual domain mask

In [ ]:
dates_out = dates[:]
ACy = {}
data= []

roi = fcGeom
for year in dates:
    sDate_T1 = str(year)+"-05-01"; 
    eDate_T1 = str(year)+"-09-30";
    #Sort by:  roi, date:
    if source == 'ls':
        collection= ls \
            .filterBounds(roi) \
            .sort('system:start', True) \
            .filterDate(sDate_T1,eDate_T1)
    elif source == 's2':
        collection= s2 \
            .filterBounds(roi) \
            .sort('system:start', True) \
            .filterDate(sDate_T1,eDate_T1)

    # Create a list of image objects.
    imageList  = collection.toList(100);
    Collection = collection.map(ndviMap).map(mndwiMap).map(RGBtoHSV)
    median     = Collection.reduce(ee.Reducer.median())
    maxi       = Collection.reduce(ee.Reducer.percentile([90]))
    mini       = Collection.reduce(ee.Reducer.min())

    ndvimap_median  = median.select('ndvi_median').clip(roi)
    ndvimap_90p     = maxi.select('ndvi_p90').clip(roi)
    mndwimap_median = median.select('mndwi_median').clip(roi)
    satmap_90p      = maxi.select('saturation_p90').clip(roi)
    ndwimap_90p     = maxi.select('mndwi_p90').clip(roi)

    pm_mndwi_0_3 = peronaMalikFilter(mndwimap_median, 5, 2, 1, 0.3)
    otsu_mndwi   = otsu(histogram(mndwimap_median, scale_hist).get('mndwi_median'  ))
    otsu_sat     = otsu(histogram(satmap_90p     , scale_hist).get('saturation_p90'))
    otsu_mndwiPM = otsu(histogram(pm_mndwi_0_3   , scale_hist).get('mndwi_median'  ))
    
    veg1 = ndvimap_90p.select('ndvi_p90').gte(0.15)
    water3 =  satmap_90p.gt(otsu_sat).Or(ee.Image(mndwimap_median.select('mndwi_median')
                                                  .gte(otsu_mndwi)).And(veg1.lt(1)))

    waterPM3 = satmap_90p.gt(otsu_sat).Or(pm_mndwi_0_3.select('mndwi_median')
                                          .gte(otsu_mndwiPM)).And(veg1.lt(1))

    area_raw_N = areaImg(water3.  remap(ee.List([0]),ee.List([1])), scale)
    area_pm_N  = areaImg(waterPM3.remap(ee.List([0]),ee.List([1])), scale)

    if source == 's2': 
        if scaledLS:
            pm_mndwi_0_3_scaledLS = peronaMalikFilter(mndwimap_median, 5, 6, 1, 0.3)
            otsu_mndwiPM_scaledLS = otsu(histogram(pm_mndwi_0_3_scaledLS, scale_hist).get('mndwi_median'))
            waterPM3_scaledLS     = satmap_90p.gt(otsu_sat).Or(pm_mndwi_0_3_scaledLS.select('mndwi_median').gte(otsu_mndwiPM_scaledLS)).And(veg1.lt(1))
            area_pm_N_scaledLS = areaImg(waterPM3_scaledLS.remap(ee.List([0]),ee.List([1])), scale)
        
    try:
        # extract the value as a number
        area_raw_number = area_raw_N.getNumber('remapped').getInfo()
        area_pm_number  = area_pm_N .getNumber('remapped').getInfo()
        # dataframe with extracted areas and thresholds info creation and compiling
        if source == 's2': 
            if scaledLS:
                area_pm_number_scaledLS = area_pm_N_scaledLS.getNumber('remapped').getInfo()
                data.append(dict(zip(('year','area_raw_number','area_pm_number','area_pm_number_scaledLS','%d','t_nmndwi','t_PMmndwi'),
                            (str(year),area_raw_number,area_pm_number,area_pm_number_scaledLS,(100*((area_raw_number-area_pm_number)/area_raw_number)),otsu_mndwi.getInfo(),otsu_mndwiPM.getInfo(),))))
            else:    
                data.append(dict(zip(('year','area_raw_number','area_pm_number','%d','t_nmndwi','t_PMmndwi'),
                        (str(year),area_raw_number,area_pm_number,(100*((area_raw_number-area_pm_number)/area_raw_number)),otsu_mndwi.getInfo(),otsu_mndwiPM.getInfo(),))))
        else:
            data.append(dict(zip(('year','area_raw_number','area_pm_number','%d','t_nmndwi','t_PMmndwi'),
                        (str(year),area_raw_number,area_pm_number,(100*((area_raw_number-area_pm_number)/area_raw_number)),otsu_mndwi.getInfo(),otsu_mndwiPM.getInfo(),))))
        
        
        #Export the images, specifying scale and region.
        task = ee.batch.Export.image.toDrive(**{
                'image': waterPM3.clip(fcGeom),
                'description': river_name + '_' + str(year),
                'folder': drive_folder,
                'scale': scale,
                'crs': EPSG_code,
                'region': fcGeom

            })
        task.start()

        if source == 's2': 
            if scaledLS:
                task = ee.batch.Export.image.toDrive(**{
                        'image': waterPM3_scaledLS.clip(fcGeom),
                        'description': 'scaledLS_' + river_name + str(year),
                        'folder': drive_folder,
                        'scale': scale,
                        'crs': EPSG_code,
                        'region': fcGeom

                    })
                task.start()
                    
        print(str(year) + ' Done!')
    except Exception as error:
        if print_errors:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero
        print(str(year) + ' Error!')
        dates_out.remove(year)

print('Years where there are data:')
print(dates_out)

In [ ]:
# saving the dataframe as csv file
df = pd.DataFrame(data)
df.to_csv(output_folder + river_name + '_' + source + '_stats.csv')
df

In [ ]:
tasks_list = ee.data.listOperations()
run, not_run = check_tasks_status(tasks_list)

while run != 0 or not_run != 0:
    print('There are still %d running tasks and %d task to start. Cheching again in a minute.' %(run, not_run))    
    time.sleep(download_wait)
    run, not_run = check_tasks_status()


# download the files from Google Drive and delete them afterwards
# list all folders in the root Drive folder
folder_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

if not folder_list:
    print('No folders found in the root folder')
else:
    print('Downloading files from Google Drive')

    # iterate over all folders
    for folder in folder_list:
        if not folder['title'] == drive_folder:
            if delete_other:
                print('Deleting folder: %s, id: %s' % (folder['title'], folder['id']))
                folder.Delete()
                continue
            else:
                print('Skipping folder: %s, id: %s' % (folder['title'], folder['id']))
                continue
        
        saving_folder = output_folder + 'rivermask_' + source +  '/'
        os.makedirs(saving_folder, exist_ok=True)
        print('\ntitle: %s, id: %s' % (folder['title'], folder['id']))

        # list and download all files in the folder
        file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % folder['id']}).GetList()

        for file in file_list:
            filename = file['title']
            print('title: %s, id: %s' % (file['title'], file['id']))
            
            # download file into working directory (in this case a tiff-file)
            file.GetContentFile(saving_folder + filename, mimetype="image/tiff")

            # delete file afterwards to keep the Drive empty
            file.Delete()
            
        folder.Delete()